# Prospect + specutils + SPARCL

Project: Obtain SDSS spectra using the NOIRLab SPARCL spectrum service, convert data to specutils objects as needed, and use Prospect to display the data.

In [2]:
import os
import sys
sys.path.insert(0, os.path.join(os.environ['HOME'], 'Documents', 'Code', 'git', 'desihub', 'prospect', 'py'))
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.nddata import InverseVariance
from specutils import __version__ as specutils_version, Spectrum1D, SpectrumCollection, SpectrumList
# from prospect.specutils import read_spPlate, read_spZbest
from prospect import __version__ as prospect_version
from prospect.viewer import plotspectra
from sparcl import __version__ as sparcl_version
import sparcl.client
from dl import queryClient as qc, storeClient as sc, specClient as spec
print(f"specutils=={specutils_version}")
print(f"prospect=={prospect_version}")
print(f"sparcl=={sparcl_version}")

specutils==1.9.1
prospect==1.2.5.dev503
sparcl==1.0.0


In [3]:
client = sparcl.client.SparclClient(url='https://sparclstage.datalab.noirlab.edu') # using the STAGE machine
client

(sparclclient:1.0.0, api:8.0, https://sparclstage.datalab.noirlab.edu/sparc, verbose=False, connect_timeout=1.1, read_timeout=5400.0)

In [ ]:
qc.set_profile('db01')

In [10]:
outfields = ['sparcl_id', 'ra', 'dec', 'redshift', 'spectype', 'data_release', 'redshift_err']
constraints = {'spectype': ['GALAXY'],
              'redshift': [0.5, 0.9],
              'data_release': ['DESI-EDR']}
desi_galaxies = client.find(outfields=outfields, constraints=constraints, limit=50)
["'{data_release}', '{sparcl_id}', {ra:f}, {dec:f}, {redshift:f}".format(**f) for f in desi_galaxies.records]

["'DESI-EDR', '000001ce-1432-4e38-9609-2b579a01f079', 165.061429, 83.098389, 0.813519",
 "'DESI-EDR', '000025d7-d443-47d4-8b86-96eb5a6ecc42', 186.170259, 45.660747, 0.558836",
 "'DESI-EDR', '00003a53-dba4-4e7e-ba20-33d1d0fa8cbc', 240.298043, 14.166304, 0.886934",
 "'DESI-EDR', '00007875-1c56-4f84-9680-9acc886a4748', 166.646747, 72.216338, 0.548366",
 "'DESI-EDR', '0000a1f4-c8a8-4fc1-b291-e935d459e711', 267.219213, 65.499536, 0.702347",
 "'DESI-EDR', '0001045b-f9bc-4a51-ba26-057e922abe28', 29.373732, 5.974278, 0.777364",
 "'DESI-EDR', '0001988d-2285-4ec6-a11b-868253a91ca5', 217.196861, 51.246553, 0.709187",
 "'DESI-EDR', '00019d35-8acd-42da-b839-9e8fa414b492', 216.990033, 53.931815, 0.761380",
 "'DESI-EDR', '00019ee5-60ba-47d4-a438-2e29747312b5', 243.476104, 56.216303, 0.502890",
 "'DESI-EDR', '0001b225-310a-44dd-be5d-5a190eadd103', 178.962949, 0.376801, 0.800818",
 "'DESI-EDR', '000207e8-fcc8-4bc4-8dde-24a8d60a5817', 124.955631, 33.764301, 0.543643",
 "'DESI-EDR', '00020844-cbac-4332-b

In [13]:
uuid_list = [f['sparcl_id'] for f in desi_galaxies.records]
include = ['sparcl_id', 'specid', 'data_release', 'redshift', 'flux', 'wavelength', 'model', 'ivar', 'mask', 'spectype']
desi_spectra = client.retrieve(uuid_list=uuid_list, include=include, dataset_list=['DESI-EDR'])
desi_spectra.info

{'status': {'success': True,
  'info': ["Successfully found 50 records in dr_list=['DESI-EDR']"],
  'warnings': []},
 'Count': 50}

In [19]:
assert all([(desi_spectra.records[k]['wavelength'] == desi_spectra.records[0]['wavelength']).all() for k in range(len(desi_spectra.records))])

In [30]:
desi_spectra.records[0]

{'data_release': 'DESI-EDR',
 'spectype': 'GALAXY',
 'redshift': 0.813519422288805,
 'specid': 39633563649312398,
 'mask': array([0, 0, 0, ..., 0, 0, 0]),
 'ivar': array([  0.44933867,   0.56658721,   0.54383731, ...,  84.21394348,
        120.00515747, 193.18800354]),
 'wavelength': array([3600. , 3600.8, 3601.6, ..., 9822.4, 9823.2, 9824. ]),
 'model': array([0.09361108, 0.1246717 , 0.12952064, ..., 0.46946484, 0.44035923,
        0.31227323]),
 'flux': array([ 1.0961802 , -1.68807256,  2.3059423 , ...,  0.51836067,
         0.58768326,  0.6925379 ]),
 'sparcl_id': '000001ce-1432-4e38-9609-2b579a01f079',
 '_dr': 'DESI-EDR'}

In [33]:
flux = np.zeros((len(desi_spectra.records), desi_spectra.records[0].flux.shape[0]), dtype=desi_spectra.records[0].flux.dtype)
model = np.zeros((len(desi_spectra.records), desi_spectra.records[0].model.shape[0]), dtype=desi_spectra.records[0].model.dtype)
uncertainty = np.zeros((len(desi_spectra.records), desi_spectra.records[0].ivar.shape[0]), dtype=desi_spectra.records[0].ivar.dtype)
mask = np.zeros((len(desi_spectra.records), desi_spectra.records[0].flux.shape[0]), dtype=desi_spectra.records[0].mask.dtype)
redshift = np.zeros(desi_spectra.records[0].wavelength.shape, dtype=desi_spectra.records[0].wavelength.dtype)
specid = np.zeros(desi_spectra.records[0].wavelength.shape, dtype=np.int64)
sparcl_id = list()
data_release = list()
spectype = list()
for k in range(len(desi_spectra.records)):
    flux[k, :] = desi_spectra.records[k].flux
    model[k, :] = desi_spectra.records[k].model
    uncertainty[k, :] = desi_spectra.records[k].ivar
    mask[k, :] = desi_spectra.records[k].mask
    redshift[k] = desi_spectra.records[k].redshift
    specid[k] = desi_spectra.records[k].specid
    sparcl_id.append(desi_spectra.records[k].sparcl_id)
    data_release.append(desi_spectra.records[k].data_release)
    spectype.append(desi_spectra.records[k].spectype)
meta = {'model': model * 10**-17 * u.Unit('erg cm-2 s-1 AA-1'), 'specid': specid, 'sparcl_id': sparcl_id, 'data_release': data_release, 'spectype': spectype}
desi_specutils = Spectrum1D(spectral_axis=desi_spectra.records[0].wavelength*u.AA,
                            flux=flux * 10**-17 * u.Unit('erg cm-2 s-1 AA-1'),
                            uncertainty=InverseVariance(uncertainty * 10**34 * u.Unit('erg-2 cm4 s2 AA2')),
                            redshift=redshift,
                            mask=mask,
                            meta=meta)
desi_specutils

<Spectrum1D(flux=<Quantity [[ 1.09618020e-17, -1.68807256e-17,  2.30594230e-17, ...,
             5.18360674e-18,  5.87683260e-18,  6.92537904e-18],
           [ 2.45113373e-17, -2.09138250e-17,  1.54428601e-18, ...,
             1.78548741e-17,  2.00291085e-17,  1.33881021e-17],
           [-7.37097502e-17, -4.11386871e-17, -3.39210415e-17, ...,
            -3.69320512e-18,  3.51228565e-18, -8.77717212e-19],
           ...,
           [ 1.67524600e-17,  2.58122540e-16,  1.40271301e-16, ...,
             2.18142509e-17, -6.90850019e-19, -1.82365343e-18],
           [-1.08645141e-17, -2.92329371e-18,  3.55715108e-17, ...,
            -1.21693976e-18, -2.07465231e-18, -3.91007453e-18],
           [-3.40203738e-17,  1.58873165e-17,  4.13492680e-17, ...,
            -4.24592271e-19, -2.67335922e-18,  4.86461781e-19]] erg / (Angstrom cm2 s)>, spectral_axis=<SpectralAxis 
   (observer to target:
      radial_velocity=[159991.68640988 124984.8057275  168319.57409111 ...      0.
      0.      

In [29]:
plotspectra(desi_specutils, zcatalog=None, redrock_cat=None, notebook=True)

AttributeError: 'Spectrum1D' object has no attribute 'num_spectra'

In [28]:
help(plotspectra)

Help on function plotspectra in module prospect.viewer:

plotspectra(spectra, zcatalog=None, redrock_cat=None, notebook=False, html_dir=None, title=None, with_imaging=True, with_noise=True, with_thumb_tab=True, with_vi_widgets=True, top_metadata=None, vi_countdown=-1, with_thumb_only_page=False, with_coaddcam=True, mask_type='DESI_TARGET', model_from_zcat=True, model=None, num_approx_fits=None, with_full_2ndfit=True, template_dir=None, archetype_fit=False, archetypes_dir=None, std_template_file=None)
    Main prospect routine. From a set of spectra, creates a bokeh document
    used for VI, to be displayed as an HTML page or within a Jupyter notebook.
    
    Parameters
    ----------
    spectra : :class:`~desispec.spectra.Spectra` or :class:`~specutils.Spectrum1D` or :class:`~specutils.SpectrumList` or list of :class:`~desispec.frame.Frame`
        Input spectra. :class:`~specutils.Spectrum1D` are assumed to be SDSS/BOSS/eBOSS.
        Otherwise DESI spectra or frames is assumed.
  

## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spPlate files have a common wavelength solution, so a spPlate file can be represented by a Spectrum1D object.

In [ ]:
# os.environ['SPECTRO_REDUX'] = os.path.join(os.environ['HOME'], 'Documents', 'Data', 'sdss', 'dr16', 'sdss', 'spectro', 'redux')
os.environ['SPECTRO_REDUX'] = 'sdss_dr16://' + os.path.join('sdss', 'spectro', 'redux')
run2d = '26'
plate = '2955'
mjd = '54562'
sdss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
sdss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spZbest-{plate}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

In [ ]:
run2d = 'v5_13_0'
plate = '9599'
mjd = '58131'
eboss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
eboss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, run2d, f'spZbest-{plate}-{mjd}.fits')
print(eboss_spectra)
print(eboss_redshifts)

## SDSS Spectra

In [ ]:
%%time
sdss = read_spPlate(sc.get(sdss_spectra, mode='fileobj'), limit=50)
sdss

In [ ]:
%%time
# sdss_spectra_ids = spec.query(plate=plate, ...)
# sdss = spec.getSpec(sdss_spectra_ids, fmt='spectrum1d')
# sdss

In [ ]:
sdss_z, sdss_model = read_spZbest(sc.get(sdss_redshifts, mode='fileobj'), limit=50)

In [ ]:
plotspectra(sdss, zcatalog=sdss_z, model=(sdss_model.spectral_axis.value, sdss_model.flux.value),
            notebook=True, title=os.path.basename(sdss_spectra),
            model_from_zcat=False, with_coaddcam=False, mask_type='PRIMTARGET', with_thumb_tab=False, with_vi_widgets=False)

## eBOSS Spectra

In [ ]:
%%time
eboss = read_spPlate(sc.get(eboss_spectra, mode='fileobj'), limit=50)
eboss_z, eboss_model = read_spZbest(sc.get(eboss_redshifts, mode='fileobj'), limit=50)
eboss

In [ ]:
plotspectra(eboss, zcatalog=eboss_z, model=(eboss_model.spectral_axis.value, eboss_model.flux.value),
            notebook=True, title=os.path.basename(eboss_spectra),
            model_from_zcat=False, with_coaddcam=False, mask_type='EBOSS_TARGET1', with_thumb_tab=False, with_vi_widgets=False)